In [ ]:
import graphlab

In [2]:
reviews = graphlab.load_sframe('review')

This non-commercial license of GraphLab Create is assigned to znasim@khi.iba.edu.pk and will expire on February 01, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-21 00:24:50,173 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\znasim\AppData\Local\Temp\graphlab_server_1463772288.log.0


In [3]:
reviews.num_rows()

2225213

## Joining Review Data with Business Data

In [5]:
business = graphlab.SFrame.read_json('yelp_academic_dataset_business.json',orient='lines')

Finished parsing file C:\Users\znasim\Documents\YelpNotebooks\yelp_academic_dataset_business.json

Parsing completed. Parsed 100 lines in 1.032 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 59239 lines. Lines per second: 40381

Finished parsing file C:\Users\znasim\Documents\YelpNotebooks\yelp_academic_dataset_business.json

Parsing completed. Parsed 77445 lines in 1.586 secs.

In [6]:
business.head()

attributes,business_id,categories,city
"{'Take-out': 'true','Drive-Thru': 'false', ...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg
"{'Happy Hour': 'true','Accepts Credit Cards': ...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg
{},3eu6MEFlq2Dg7bQh8QbdOg,"[Auto Repair, Automotive]",Dravosburg
{'Good for Kids': 'true'},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf,Golf] ...",Bethel Park
"{'Accepts Credit Cards':'false', 'Price Range': ...",HZdLhv6COCleJMo7nPl-RA,"[Shopping, Home Services,Internet Service ...",Pittsburgh
"{'Alcohol': 'full_bar','Noise Level': 'avera ...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New),Nightlife, Lounges, ...",rankin
{'Good for Kids':'false'} ...,VHYT1qgQw5JvXAlUDhTArA,"[Active Life, Trainers,Fitness & Instruction] ...",Pittsburgh
"{'Alcohol': 'full_bar','Noise Level': 'loud', ...",KayYbHCt-RkbGcPdGOThNg,"[Bars, American(Traditional), Nightl ...",Carnegie
{},b12U9TFESStdy7CsTtcOeg,"[Auto Repair, Automotive,Tires] ...",Carnegie
{'Good for Kids': 'true'},Sktj1eHQFuVa-M4bgnEh8g,"[Active Life, Mini Golf]",Carnegie


In [7]:
business.num_rows()

77445

In [26]:
review_business = reviews.join(business, how='inner', on='business_id')

In [29]:
# Saving joined table
review_business.save('review_business')

In [30]:
review_business.num_rows()

2225213

## Data Partition into Training and Testing Set

In [31]:
train_rev,test_rev = review_business.random_split(0.6,seed = 0)

## Data Cleaning

In [32]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
import re
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ",raw_review) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    cleanwords = []
    for word in words:
        if(word not in stopwords and len(word) > 2):
            cleanwords.append(word)
    return( " ".join( cleanwords ))   

In [33]:
train_rev['preprocessedText'] = train_rev['text'].apply(review_to_words)
test_rev['preprocessedText'] = test_rev['text'].apply(review_to_words)

## Tokenization

In [34]:
reviews_train = graphlab.text_analytics.count_words(train_rev['preprocessedText'])


In [35]:
reviews_test = graphlab.text_analytics.count_words(test_rev['preprocessedText'])


## Learning Topic Model

In [36]:
# Initializing Word topic association for seeding topic model
associations = graphlab.SFrame({'word':['food', 'italian', 'pasta', 'pizza','service','staff','worker'
                                       ,'price','money','lunch','salad'],
                                'topic': [1, 2, 2, 2,3,3,3,4,4,5,5]})

In [37]:
model = graphlab.topic_model.create(reviews_train,num_topics= 50,num_iterations=5000,
                                   associations=associations)

Learning a topic model

Number of documents   1335067

Vocabulary size    300661

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 4970      | 10h 52m       | 7.41249e+006   | 0               |

| 4980      | 10h 53m       | 7.90286e+006   | 0               |

| 4990      | 10h 54m       | 7.77878e+006   | 0               |

| 5000      | 10h 56m       | 8.50348e+006   | 0               |

+-----------+---------------+----------------+-----------------+

## Saving Topic Model

In [38]:
model.save('my_topicModel_50topics_5000iters')

In [39]:
print model

Class                         : TopicModel

Schema
------
Vocabulary Size               : 300661

Settings
--------
Number of Topics              : 50
alpha                         : 1.0
beta                          : 0.1
Iterations                    : 5000
Training time                 : 39367.6915
Verbose                       : False

Accessible fields             : 
m['topics']                   : An SFrame containing the topics.
m['vocabulary']               : An SArray containing the words in the vocabulary.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.


In [40]:
print model['topics']

+-------------------------------+------------+
|      topic_probabilities      | vocabulary |
+-------------------------------+------------+
| [6.37830722408e-08, 0.4918... |    food    |
| [6.37830722408e-08, 6.9383... |  italian   |
| [6.37830722408e-08, 6.9383... |   pasta    |
| [6.37830722408e-08, 6.9383... |   pizza    |
| [6.37830722408e-08, 6.9383... |  service   |
| [6.37830722408e-08, 6.9383... |   staff    |
| [6.37830722408e-08, 6.9383... |   worker   |
| [6.37830722408e-08, 6.9383... |   price    |
| [6.37830722408e-08, 6.9383... |   money    |
| [6.37830722408e-08, 6.9383... |   lunch    |
+-------------------------------+------------+
[300661 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [95]:
print model['vocabulary']

['hole', 'key', 'low', 'carnegie', 'gem', 'wings', 'atmosphere', 'prices', 'selection', 'wall', 'relaxing', 'beer', 'good', 'buffalo', 'greetree', 'friendly', 'area', 'staff', 'chicken', 'food', 'hidden', 'place', 'pizza', 'thick', 'sandwiches', 'cut', 'fresh', 'italian', 'great', 'bread', 'system', 'fuel', 'reading', 'refused', 'wheel', 'manual', 'brakes', 'release', 'walked', 'mechanic', 'signal', 'glove', 'windshield', 'gas', 'owners', 'quicker', 'cow', 'gear', 'turned', 'back', 'hazard', 'needed', 'put', 'triple', 'present', 'hands', 'figured', 'learn', 'show', 'rear', 'lot', 'customer', 'lucky', 'box', 'leather', 'case', 'head', 'clean', 'car', 'embarrassment', 'idea', 'wipers', 'jack', 'means', 'proud', 'steering', 'day', 'extra', 'dealership', 'saved', 'defoggers', 'advanced', 'driving', 'training', 'auto', 'rain', 'god', 'parking', 'gave', 'cover', 'send', 'sounded', 'involved', 'putting', 'skinned', 'alive', 'time', 'didn', 'isn', 'associate', ... ]


In [41]:
model.get_topics().print_rows(500)

+-------+------------+------------------+
| topic |    word    |      score       |
+-------+------------+------------------+
|   0   |    care    | 0.0202658593261  |
|   0   |    dog     | 0.0174976739909  |
|   0   |   office   | 0.0155274148894  |
|   0   |    dogs    | 0.00937617540247 |
|   0   |    time    | 0.0087166584355  |
|   1   |    food    |  0.491885243793  |
|   1   |    good    | 0.0540487189969  |
|   1   | restaurant | 0.0397334382603  |
|   1   |   place    | 0.0334424024694  |
|   1   |   great    | 0.0260884057397  |
|   2   |   pizza    | 0.0892770952442  |
|   2   |    good    | 0.0305262760738  |
|   2   |   cheese   | 0.0208769276401  |
|   2   |   sauce    | 0.0205420650124  |
|   2   |  italian   | 0.0203033510599  |
|   3   |  service   |  0.341991735064  |
|   3   |   staff    |  0.147183860652  |
|   3   |  friendly  | 0.0769128231989  |
|   3   |   great    | 0.0639572035212  |
|   3   |  customer  | 0.0255653742063  |
|   4   |   price    | 0.081360374

## Predicting Topic on Review Set

In [69]:
test_rev_topics = model.predict(reviews_test,output_type = "probability")

In [140]:
train_rev['test_rev_topics'] = model.predict(reviews_train,output_type = "probability")

In [71]:
test_rev['test_rev_topics'] = test_rev_topics

In [141]:
train_rev = train_rev.add_row_number()

In [135]:
test_rev = test_rev.add_row_number()

In [136]:
test_rev.head()

id,business_id,date,review_id,stars,text,type
0,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superbcustomer service. I miss ...,review
1,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is alittle out dated and not ...,review
2,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is greathere. But the best thing ...,review
3,UsFtqoBl7naz8AVUBZMjQQ,2014-10-29,7N9j5YbBHBW6qguE5DAeyA,1,Wing sauce is like water.Pretty much a lot of ...,review
4,cE27W9VPgO88Qxe4ol6y_g,2013-08-16,iS34GJhMkkt9kCoTJLYEwA,4,Before I finally made itover to this range I ...,review
5,HZdLhv6COCleJMo7nPl-RA,2013-06-10,fBQ69-NU9ZyTjjS7Tb5tww,5,THANK YOU ROB! i trulyappreciated all the help ...,review
6,HZdLhv6COCleJMo7nPl-RA,2013-12-23,CFiLh7WvH7dM3qVZvNiacQ,2,After waiting for almost30 minutes to trade i ...,review
7,mVHrayjG3uZ_RLHkLj-AMg,2013-03-30,3Es8GsjkssusYgeU6_ZVpQ,5,This place should have alot more reviews - but ...,review
8,mVHrayjG3uZ_RLHkLj-AMg,2014-05-01,PY6nIMMLP2pqARlbIOJuRA,5,"Like a lot of recentpatrons, I heard about ...",review
9,mVHrayjG3uZ_RLHkLj-AMg,2014-09-29,5uyYmniYyIB_wtKtyXDudQ,4,Wonderful reuben. Mapshown on Yelp page is ...,review


In [143]:
sf_train = graphlab.SFrame({'predictions': train_rev['test_rev_topics']}).add_row_number()

In [80]:
# Stack these predictions and add a row number.
sf = graphlab.SFrame({'predictions': test_rev_topics}).add_row_number()

def top2_dict(lst):
    """ Return the top 2 elements of a list along with the indices and then
        convert the result into a dictionary. For example:

        input  : [10, 5, 8, 20]
        output : {0: 20, 3: 10} 
    """
    sorted_lst = sorted(enumerate(lst), key=lambda x: x[1], reverse=True)
    # Return the top2
    return dict(sorted_lst[:2])


In [81]:
# Top-2 predictions
sf['predictions'] = sf['predictions'].apply(top2_dict)

In [144]:
# Top-2 predictions
sf_train['predictions'] = sf_train['predictions'].apply(top2_dict)

In [145]:
sf_train = sf_train.stack('predictions', ['topic', 'probability'])
sf_train.head()

id,topic,probability
0,48,0.0568181818182
0,26,0.113636363636
1,6,0.0714285714286
1,47,0.160714285714
2,5,0.1
2,38,0.1
3,33,0.051724137931
3,49,0.051724137931
4,49,0.118644067797
4,17,0.0762711864407


In [83]:
sf = sf.stack('predictions', ['topic', 'probability'])
sf.head()

id,topic,probability
0,33,0.0625
0,3,0.0625
1,40,0.0595238095238
1,5,0.0595238095238
2,9,0.0659340659341
2,47,0.0989010989011
3,14,0.0588235294118
3,47,0.247058823529
4,4,0.100775193798
4,20,0.108527131783


## Assigning Meaningful Label to Topics

In [121]:
def addLabel(x):
    if(x == 1 or x==26 or x==35 or x==46 or x==47):
        return "Food"
    elif(x == 2):
        return "Italian"
    elif(x == 3 or x==19 or x==30):
        return "Service"
    elif(x == 4):
        return "Price"
    elif(x == 5):
        return "Lunch"
    elif(x == 9 or x==6 or x==18 or x==20 or x== 28 or x==32 or x==33 or x==39):
        return "Location"
    elif(x == 8):
        return "Club"
    elif(x == 12):
        return "Chinese Food"
    elif(x == 17):
        return "Experience"
    elif(x == 22):
        return "Store"
    elif(x == 25 or x==36):
        return "Time"
    elif(x == 31 or x==34 or x==48):
        return "Ambiance"
    elif(x == 38):
        return "Drink"
    elif(x == 40):
        return "Desserts"
    elif(x == 41):
        return "SeaFood"
    elif(x == 43):
        return "Breakfast"
    elif(x == 44):
        return "Mexican Food"
    elif(x == 45):
        return "Beverages"
    elif(x == 49):
        return "Car"
    else:
        return "Misc"

In [146]:
sf_train['topicLabel'] = sf_train['topic'].apply(addLabel)

In [122]:
sf['topicLabel'] = sf['topic'].apply(addLabel)

In [127]:
sf.column_names()

['id', 'topic', 'probability', 'topicLabel']

In [147]:
sff_train = sf_train.groupby('id',{'topic' : graphlab.aggregate.CONCAT('topic'),
                       'topicLabel':graphlab.aggregate.CONCAT('topicLabel')})

In [128]:
sff = sf.groupby('id',{'topic' : graphlab.aggregate.CONCAT('topic'),
                       'topicLabel':graphlab.aggregate.CONCAT('topicLabel')})

In [148]:
train_dataset = train_rev.join(sff_train,on = "id",how = "inner")

In [149]:
train_dataset.num_rows()

1335067

In [150]:
train_dataset.head()

id,business_id,date,review_id,stars,text,type
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is aninstitution. Walking in, ...",review
1,UsFtqoBl7naz8AVUBZMjQQ,2014-03-29,0Lua2-PbqEQMjD9r89-asw,3,We checked this place outthis past Monday for ...,review
2,UsFtqoBl7naz8AVUBZMjQQ,2014-11-28,mjCJR33jvUNt41iJCxDU_g,4,Cold cheap beer. Good barfood. Good service. ...,review
3,3eu6MEFlq2Dg7bQh8QbdOg,2014-02-27,Ieh3kfZ-5J9pLju4JiQDvQ,5,I highly recommend thisplace. The mechanics are ...,review
4,3eu6MEFlq2Dg7bQh8QbdOg,2015-06-16,PU28OoBSHpZLkYGCmNxlmg,5,"I am a big believer infirst impressions, so ...",review
5,cE27W9VPgO88Qxe4ol6y_g,2012-08-19,XsA6AojkWjOHA4FmuAb8XQ,3,Decent range somewhatclose to the city. The ...,review
6,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,rkD7UDbQ9VM3Va6bI-eBHQ,1,Owning a driving rangeinside the city limit ...,review
7,cE27W9VPgO88Qxe4ol6y_g,2013-07-14,WExNE-f93SL4D1q8s9QWKg,1,This place is absolutegarbage... Half of the ...,review
8,cE27W9VPgO88Qxe4ol6y_g,2014-07-11,S-G0D8Cy7PnqShoBZu8PCA,4,I drove by yesterday toget a sneak peak. It ...,review
9,HZdLhv6COCleJMo7nPl-RA,2014-09-04,UzMViMQZuSxOr5wrru3LwQ,4,I visited this storeseveral months ago to ...,review


In [137]:
test_dataset = test_rev.join(sff,on = "id",how = "inner")

In [139]:
test_dataset.head()

id,business_id,date,review_id,stars,text,type
0,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superbcustomer service. I miss ...,review
1,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is alittle out dated and not ...,review
2,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is greathere. But the best thing ...,review
3,UsFtqoBl7naz8AVUBZMjQQ,2014-10-29,7N9j5YbBHBW6qguE5DAeyA,1,Wing sauce is like water.Pretty much a lot of ...,review
4,cE27W9VPgO88Qxe4ol6y_g,2013-08-16,iS34GJhMkkt9kCoTJLYEwA,4,Before I finally made itover to this range I ...,review
5,HZdLhv6COCleJMo7nPl-RA,2013-06-10,fBQ69-NU9ZyTjjS7Tb5tww,5,THANK YOU ROB! i trulyappreciated all the help ...,review
6,HZdLhv6COCleJMo7nPl-RA,2013-12-23,CFiLh7WvH7dM3qVZvNiacQ,2,After waiting for almost30 minutes to trade i ...,review
7,mVHrayjG3uZ_RLHkLj-AMg,2013-03-30,3Es8GsjkssusYgeU6_ZVpQ,5,This place should have alot more reviews - but ...,review
8,mVHrayjG3uZ_RLHkLj-AMg,2014-05-01,PY6nIMMLP2pqARlbIOJuRA,5,"Like a lot of recentpatrons, I heard about ...",review
9,mVHrayjG3uZ_RLHkLj-AMg,2014-09-29,5uyYmniYyIB_wtKtyXDudQ,4,Wonderful reuben. Mapshown on Yelp page is ...,review


In [132]:
test_rev.num_rows()

890146

In [133]:
sff[sff['id'] == 0]

id,topic,topicLabel
0,"[33, 3]","[Location, Service]"


In [55]:
train_rev['test_rev_topics'] = model.predict(reviews_train)

In [155]:
del reviews_business

In [156]:
reviews_business_topiclabels = train_dataset.append(test_dataset)

In [158]:
reviews_business_topiclabels.num_rows()

2225213

In [159]:
reviews_business_topiclabels.save('reviews_business_topiclabels')

## Evaluating Model Perplexity

In [54]:
model.evaluate(reviews_train,reviews_test)

{'perplexity': 3379.4400268292516}